<a href="https://colab.research.google.com/github/FlyAIBox/LLM-101/blob/main/chapter09-llm-project/langfuse/integration_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
description: 面向初学者的 LangChain（Python）× Langfuse 集成示例手册。
category: Integrations
---

## 📚 前置知识导读

在开始学习 LangChain 与 Langfuse 的集成之前，让我们先了解一些基础概念：

### 🤖 LangChain 是什么？
- **定义**：LangChain 是一个用于构建大语言模型（LLM）应用的开发框架
- **作用**：帮助开发者将 LLM 与外部数据源、工具、数据库等连接起来
- **核心概念**：Chain（链）、Agent（智能体）、Tool（工具）、Memory（记忆）等

### 📊 Langfuse 是什么？
- **定义**：Langfuse 是一个专门为大语言模型应用设计的可观测性（Observability）平台
- **作用**：帮助开发者监控、调试和优化 LLM 应用的性能
- **核心功能**：追踪（Tracing）、评估（Evaluation）、监控（Monitoring）

### 🔗 两者如何协同工作？
```mermaid
graph LR
    A[你的应用] --> B[LangChain]
    B --> C[LLM/工具/数据]
    B --> D[Langfuse]
    D --> E[性能监控]
    D --> F[调试分析]
    D --> G[成本追踪]
```

### 🎯 学习目标
通过本教程，你将学会：
1. 如何在 LangChain 应用中集成 Langfuse
2. 如何追踪和监控 LLM 调用链
3. 如何分析应用性能和成本
4. 如何调试和优化 LLM 应用

---
**💡 提示**：如果你对 LangChain 或 Langfuse 还不太熟悉，建议先阅读官方文档了解基础概念。

# 示例手册：LangChain 集成

这是一本汇集了 Langfuse 与 LangChain（Python）集成示例的“示例手册”。

请按照[集成指南](https://langfuse.com/integrations/frameworks/langchain)将该集成添加到你的 LangChain 项目中。该集成也支持 LangChain JS。

## 环境准备

In [1]:
# 📦 安装必要的 Python 包
#
# 包说明：
# - langfuse: Langfuse 客户端库，用于应用监控和追踪
# - langchain: LangChain 核心框架，用于构建 LLM 应用
# - langchain-openai: LangChain 的 OpenAI 集成包
# - langchain_community: LangChain 社区贡献的组件和工具
#
# 💡 版本说明：
# 这里指定了具体版本号以确保兼容性，在生产环境中建议锁定版本
# 如果需要最新版本，可以使用 --upgrade 参数
#
# 🔧 安装命令（二选一）：
# 方式1：指定版本（推荐，确保稳定性）
%pip install langfuse==3.3.0 langchain==0.3.27 langchain-openai==0.3.31 langchain_community==0.3.27
#
# 方式2：安装最新版本（可能存在兼容性问题）
# %pip install langfuse langchain langchain_openai langchain_community --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


在 Langfuse 控制台的项目设置页获取 API Key，初始化 Langfuse 客户端，并将其设置到环境变量中。

In [2]:
import os

# ===== Langfuse 配置 =====
# 🔑 获取 Langfuse API 密钥的步骤：
# 1. 访问 https://cloud.langfuse.com 注册账号
# 2. 创建新项目或选择现有项目
# 3. 在项目设置页面找到 API Keys 部分
# 4. 复制 Public Key 和 Secret Key

# Langfuse 公钥（用于标识项目）
# 格式：pk-lf-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-2cf6d992-461d-4b3c-a47e-96285c1e5526"

# Langfuse 私钥（用于身份验证，请妥善保管）
# 格式：sk-lf-xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-55c2fa56-c913-44be-916b-85ba498dcb8a"

# Langfuse 服务器地址（根据你的地理位置选择）
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"  # 🇪🇺 欧盟区域（默认）
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"  # 🇺🇸 美国区域

# ===== OpenAI 配置 =====
# 💡 提示：如果你在中国大陆，可能需要使用代理服务
# 这里使用的是一个国内代理服务示例

# OpenAI API 密钥
# 🔑 获取方式：
# - 官方：https://platform.openai.com/api-keys
# - 代理：https://api.apiyi.com（示例）
os.environ["OPENAI_API_KEY"] = "sk-SoCBsC2X4NvGNzBMBaDa87Ea41354179Bd3c94A627666142"

# OpenAI API 基础 URL（如果使用代理服务）
# 官方地址：https://api.openai.com/v1
# 代理地址：https://api.apiyi.com/v1（示例）
os.environ["OPENAI_BASE_URL"] = "https://api.apiyi.com/v1"

# 🚨 安全提醒：
# 1. 不要将真实的 API 密钥提交到公共代码仓库
# 2. 在生产环境中使用环境变量或密钥管理服务
# 3. 定期轮换 API 密钥以提高安全性
# 4. 监控 API 使用量，避免意外的高额费用


In [3]:
from langfuse.langchain import CallbackHandler

# 初始化 Langfuse 回调处理器
# 这个处理器会自动捕获 LangChain 的执行过程，包括：
# - LLM 调用的输入和输出
# - 执行时间和延迟
# - 错误信息（如果有）
# - 成本信息（token 使用量）
langfuse_handler = CallbackHandler()

# 💡 提示：这个回调处理器将在后续的 LangChain 调用中使用
# 通过 config={"callbacks": [langfuse_handler]} 参数传递

## 示例

### LangChain LCEL 的跟踪图

![LangChain LCEL 的跟踪图](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202508261521543.png)

[Langfuse 中的示例追踪](https://cloud.langfuse.com/project/cmequpe0j00euad07w6wrvkzg/traces?peek=9ec09bc81b4173edf6e3d11d24d0cf2f&timestamp=2025-08-26T07%3A11%3A54.389Z)

In [4]:
# 导入必要的模块
from operator import itemgetter  # 用于从字典中提取特定键值
from langchain_openai import ChatOpenAI  # OpenAI 聊天模型
from langchain.prompts import ChatPromptTemplate  # 聊天提示模板
from langchain.schema import StrOutputParser  # 字符串输出解析器

# 重新初始化回调处理器（确保追踪正常工作）
langfuse_handler = CallbackHandler()

# 第一个提示模板：询问某人来自哪座城市
prompt1 = ChatPromptTemplate.from_template("{person} 来自哪座城市？")

# 第二个提示模板：询问城市位于哪个国家，并指定回答语言
prompt2 = ChatPromptTemplate.from_template(
    "城市 {city} 位于哪个国家？请用 {language} 回答"
)

# 初始化 OpenAI 聊天模型
model = ChatOpenAI()

# 构建第一个链：人名 -> 提示1 -> 模型 -> 字符串输出
chain1 = prompt1 | model | StrOutputParser()

# 构建第二个链（复合链）：
# 1. 并行执行：chain1获取城市，itemgetter提取语言参数
# 2. 将结果传递给 prompt2
# 3. 调用模型获取最终答案
# 4. 解析为字符串输出
chain2 = (
    {"city": chain1, "language": itemgetter("language")}  # 并行处理
    | prompt2  # 第二个提示
    | model    # 模型调用
    | StrOutputParser()  # 输出解析
)

# 执行链式调用并启用 Langfuse 追踪
# 这将在 Langfuse 控制台中创建一个完整的执行追踪
chain2.invoke({"person": "苏东坡", "language": "中文"}, config={"callbacks":[langfuse_handler]})
# chain2.invoke({"person": "obama", "language": "spanish"}, config={"callbacks":[langfuse_handler]})

'苏东坡来自中国。'

#### Runnable 方法

Runnable 是可以被调用、批处理、流式处理、转换并进行组合的工作单元。

下面的示例展示了如何在 Langfuse 中使用这些方法：

- invoke/ainvoke：将单个输入转换为输出。
- batch/abatch：高效地将多个输入批量转换为输出。
- stream/astream：在生成过程中以流式方式输出单个输入的结果。

In [5]:
# 🔄 异步调用（Async Invoke）
# 适用场景：需要在异步环境中执行单个请求
# 优势：不会阻塞其他异步操作
await chain2.ainvoke({"person": "biden", "language": "german"}, config={"callbacks":[langfuse_handler]})

# 📦 批处理（Batch）
# 适用场景：需要同时处理多个相似的请求
# 优势：比逐个调用更高效，可以并行处理
chain2.batch([
    {"person": "elon musk", "language": "english"},
    {"person": "mark zuckerberg", "language": "english"}
], config={"callbacks":[langfuse_handler]})

# 🔄📦 异步批处理（Async Batch）
# 适用场景：在异步环境中批量处理请求
# 优势：结合了异步和批处理的优点
await chain2.abatch([
    {"person": "jeff bezos", "language": "english"},
    {"person": "tim cook", "language": "english"}
], config={"callbacks":[langfuse_handler]})

# 🌊 流式处理（Stream）
# 适用场景：需要实时显示生成过程，提升用户体验
# 优势：用户可以立即看到部分结果，无需等待完整响应
for chunk in chain2.stream({"person": "steve jobs", "language": "english"}, config={"callbacks":[langfuse_handler]}):
    print("流式分片:", chunk)  # 每个分片会实时输出

# 🔄🌊 异步流式处理（Async Stream）
# 适用场景：在异步环境中进行流式处理
# 优势：不阻塞其他异步操作，同时提供实时反馈
async for chunk in chain2.astream({"person": "bill gates", "language": "english"}, config={"callbacks":[langfuse_handler]}):
    print("异步流式分片:", chunk)  # 异步接收每个分片


流式分片: 
流式分片: The
流式分片:  city
流式分片:  of
流式分片:  San
流式分片:  Francisco
流式分片: ,
流式分片:  California
流式分片: ,
流式分片:  where
流式分片:  Steve
流式分片:  Jobs
流式分片:  is
流式分片:  from
流式分片: ,
流式分片:  is
流式分片:  located
流式分片:  in
流式分片:  the
流式分片:  United
流式分片:  States
流式分片: .
流式分片: 
流式分片: 
异步流式分片: 
异步流式分片: The
异步流式分片:  city
异步流式分片:  Seattle
异步流式分片: ,
异步流式分片:  where
异步流式分片:  Bill
异步流式分片:  Gates
异步流式分片:  comes
异步流式分片:  from
异步流式分片: ,
异步流式分片:  is
异步流式分片:  located
异步流式分片:  in
异步流式分片:  the
异步流式分片:  United
异步流式分片:  States
异步流式分片: .
异步流式分片: 
异步流式分片: 


### Langfuse中的LangChain检索式问答跟踪（RetrievalQA）

![Langfuse 中的 LangChain 检索式问答跟踪](https://cdn.jsdelivr.net/gh/Fly0905/note-picture@main/imag/202508261524781.png)

[Langfuse 中的示例追踪](https://cloud.langfuse.com/project/cmequpe0j00euad07w6wrvkzg/traces?peek=24338e3eec59d8e66b1de0ff7f653c36&timestamp=2025-08-26T07%3A13%3A08.424Z)

In [6]:
import os
# 设置 SerpAPI 密钥（用于网络搜索功能）
# 💡 提示：如果不使用搜索功能，可以跳过这个设置
# 获取免费 API 密钥：https://serpapi.com/
os.environ["SERPAPI_API_KEY"] = "12607ddca2f9b73a4b6624ad3a972ebfd9bf9653bda3b7b95d6de741d09dd295"

In [7]:
# 安装检索式问答所需的额外依赖包
# unstructured: 用于处理各种文档格式（PDF、Word、HTML等）
# selenium: 用于网页内容抓取和动态页面处理
# langchain-chroma: 向量数据库，用于存储和检索文档嵌入
%pip install unstructured==0.18.13 selenium==4.35.0 langchain-chroma==0.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
# 导入检索式问答所需的模块
from langchain_community.document_loaders import SeleniumURLLoader  # 网页文档加载器
from langchain_chroma import Chroma  # 向量数据库
from langchain_text_splitters import CharacterTextSplitter  # 文本分割器
from langchain_openai import OpenAIEmbeddings, ChatOpenAI  # OpenAI 嵌入模型和聊天模型
from langchain.chains import RetrievalQA  # 检索式问答链

# 初始化 Langfuse 回调处理器
langfuse_handler = CallbackHandler()

# 定义要加载的文档 URL 列表
# 这里使用的是美国国情咨文演讲稿作为示例文档
urls = [
    "https://raw.githubusercontent.com/langfuse/langfuse-docs/main/public/state_of_the_union.txt",
]

# 步骤1：加载文档
# SeleniumURLLoader 可以处理动态网页内容
loader = SeleniumURLLoader(urls=urls)

# 步骤2：初始化语言模型
# 使用 ChatOpenAI 而不是 OpenAI，因为它与现代 API 更兼容
llm = ChatOpenAI(model="gpt-3.5-turbo")

# 步骤3：加载并处理文档
documents = loader.load()  # 从 URL 加载文档内容

# 步骤4：文档分割
# 将长文档分割成较小的块，便于检索和处理
# chunk_size=1000: 每个文本块最大1000个字符
# chunk_overlap=0: 文本块之间没有重叠
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print("【检索式问答-文档分割的内容】",texts)

# 步骤5：创建向量嵌入
# 将文本转换为向量表示，用于语义搜索
embeddings = OpenAIEmbeddings()

# 步骤6：构建向量数据库
# Chroma 是一个轻量级的向量数据库，用于存储和检索文档嵌入
docsearch = Chroma.from_documents(texts, embeddings)
print("【检索式问答-向量数据库的内容】",docsearch)

# 步骤7：定义查询问题
query = "美国国情咨文演讲稿的核心主题是什么？"

# 步骤8：构建检索式问答链
# RetrievalQA 会先检索相关文档，然后基于检索到的内容回答问题
# search_kwargs={"k": 1}: 只检索最相关的1个文档块
chain = RetrievalQA.from_chain_type(
    llm,  # 语言模型
    retriever=docsearch.as_retriever(search_kwargs={"k": 3}),  # 检索器配置
)

# 步骤9：执行问答并启用 Langfuse 追踪
# 这将在 Langfuse 中记录整个检索和问答过程
chain.invoke(query, config={"callbacks":[langfuse_handler]})

【检索式问答-文档分割的内容】 [Document(metadata={'source': 'https://raw.githubusercontent.com/langfuse/langfuse-docs/main/public/state_of_the_union.txt', 'title': 'No title found.', 'description': 'No description found.', 'language': 'No language found.'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll ove

{'query': '美国国情咨文演讲稿的核心主题是什么？',
 'result': '美国国情咨文演讲稿的核心主题包括国家实力强大、挑战与困难、民众的力量与团结、经济竞争、基础设施建设、气候变化与环境正义、技术发展、工作机会创造等。演讲稿提倡民众团结一心，共同应对当前的挑战，使美国更加繁荣与强大。'}

## 🎉 总结与下一步学习

恭喜！你已经完成了 LangChain 与 Langfuse 集成的基础学习。让我们回顾一下学到的内容：

### 📋 本教程涵盖的内容
- ✅ **环境准备**：安装依赖、配置 API 密钥、初始化回调处理器
- ✅ **基础集成**：在 LangChain 应用中使用 Langfuse 进行追踪
- ✅ **LCEL 示例**：顺序链的构建与执行
- ✅ **Runnable 方法**：同步/异步、批处理、流式处理
- ✅ **检索式问答**：文档加载、向量化、问答链构建

### 🚀 下一步学习建议

#### 1. 深入 LangChain 核心概念
- 学习 [LangChain 官方教程](https://python.langchain.com/docs/tutorials/)
- 掌握 Agent（智能体）和 Tool（工具）的使用
- 了解 Memory（记忆）和 Chain（链）的高级用法

#### 2. 探索 Langfuse 高级功能
- 学习如何创建自定义评估指标
- 掌握成本分析和性能优化
- 了解团队协作和项目管理功能

#### 3. 实战项目练习
- 构建一个带有多步骤推理的聊天机器人
- 创建一个支持文档检索的问答系统
- 开发一个多模态（文本+图像）应用

#### 4. 生产环境部署
- 学习 Docker 容器化部署
- 了解 Kubernetes 集群管理
- 掌握监控和日志管理最佳实践

### 🔗 有用的资源链接

#### 📚 官方文档
- [LangChain 官方文档](https://python.langchain.com/) - 全面的 LangChain 使用指南
- [Langfuse 官方文档](https://langfuse.com/docs) - 详细的 Langfuse 功能说明
- [OpenAI API 文档](https://platform.openai.com/docs) - OpenAI API 使用指南

#### 🛠️ 开源社区
- [LangChain GitHub](https://github.com/langchain-ai/langchain) - 源码和问题讨论
- [Langfuse GitHub](https://github.com/langfuse/langfuse) - 开源版本和贡献指南
- [LangChain 模板库](https://github.com/langchain-ai/langchain/tree/master/templates) - 实用模板集合

#### 🎓 学习资源
- [LangChain 官方教程](https://python.langchain.com/docs/tutorials/) - 从入门到进阶
- [Langfuse 示例库](https://langfuse.com/docs/integrations) - 各种集成示例
- [大模型应用开发最佳实践](https://platform.openai.com/docs/guides/prompt-engineering) - 提示工程指南

### 💡 实践建议

#### 🏗️ 开发阶段
1. **从简单开始**：先构建基础的问答链，验证核心功能后再增加复杂度
2. **模块化设计**：将复杂的应用拆分为独立的组件，便于测试和维护
3. **版本控制**：使用 Git 管理代码，记录每次重要的功能变更
4. **环境隔离**：使用虚拟环境管理依赖，避免版本冲突

#### 📊 监控与优化
1. **全程追踪**：始终使用 Langfuse 追踪应用执行，建立完整的可观测性
2. **性能分析**：定期查看 Langfuse 控制台，分析响应时间和成功率
3. **成本监控**：密切关注 token 使用量和 API 调用成本，优化模型选择
4. **错误处理**：建立完善的错误处理机制，提高应用的稳定性

#### 🔄 迭代改进
1. **数据驱动**：基于 Langfuse 的分析数据，持续优化提示词和参数
2. **A/B 测试**：对比不同版本的性能，选择最优方案
3. **用户反馈**：收集真实用户的使用反馈，指导产品改进方向
4. **定期评估**：建立定期的性能评估机制，确保应用质量

### 🚨 常见问题与解决方案

#### ❓ API 密钥问题
- **问题**：API 调用失败或认证错误
- **解决**：检查环境变量设置，确保密钥正确且有效

#### ❓ 依赖包冲突
- **问题**：安装包时出现版本冲突
- **解决**：使用虚拟环境，或指定具体的包版本

#### ❓ 追踪数据缺失
- **问题**：Langfuse 控制台看不到追踪数据
- **解决**：确保回调处理器正确配置，检查网络连接

#### ❓ 性能问题
- **问题**：应用响应速度慢
- **解决**：优化提示词长度，选择合适的模型，使用缓存机制

---

### 🎯 最后的话

大模型应用开发是一个充满挑战和机遇的领域。通过本教程，你已经掌握了：

- ✅ **基础技能**：LangChain 与 Langfuse 的集成使用
- ✅ **实践经验**：从简单链到复杂应用的构建过程
- ✅ **监控能力**：使用 Langfuse 进行全面的应用监控
- ✅ **优化思路**：基于数据驱动的持续改进方法

记住，**实践是最好的老师**。建议你：
1. 🔨 **动手实践**：基于本教程的示例，构建自己的应用
2. 📖 **持续学习**：关注 LangChain 和 Langfuse 的最新发展
3. 🤝 **社区参与**：加入开发者社区，分享经验和学习心得
4. 🚀 **勇于创新**：探索大模型在你所在领域的应用可能性